
# Bayesian Multivariate Regression: Tap Rate & CV–ITI (PyMC v5)

This notebook estimates a **joint Bayesian multivariate model** for:
- **Interaction frequency**: `tap_rate_z`
- **Interaction regularity**: `cv_iti_z`

Predictors: `orientation_entropy_z`, `degree_mean_z`, `circuity_log_z`, `SA_z`, `SBSOD_z`, `Gender_M` (centered).  
Outputs are written to `notebooks/outputs/bayesmv/`.


In [ ]:

# Imports
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import pymc as pm
import pytensor.tensor as at
import arviz as az

# Repro
np.random.seed(42)

# Paths (this notebook lives in notebooks/)
DATA_PATH = Path("../data/pre-processed_anonimized_final_clean.csv")
OUT_DIR = Path("outputs") / "bayesmv"
(OUT_DIR / "figs").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "summaries").mkdir(parents=True, exist_ok=True)

print(f"Python: {os.sys.version.split()[0]}")
print("PyMC:", pm.__version__, "| ArviZ:", az.__version__)
print("Data path:", DATA_PATH.resolve() if DATA_PATH.exists() else DATA_PATH)
print("Outputs →", OUT_DIR.resolve())


In [ ]:

# Load dataset
df = pd.read_csv(DATA_PATH)

# Create Gender_M if needed
if "Gender_M" not in df.columns and "Gender" in df.columns:
    df["Gender_M"] = (df["Gender"].astype(str).str.upper().str[0] == "M").astype(float)

# Standardize SA_Score and SBDS_score into SA_z and SBSOD_z if needed
if "SA_z" not in df.columns or "SBSOD_z" not in df.columns:
    needed = {"SA_Score", "SBDS_score"}
    if needed.issubset(df.columns):
        scaler = StandardScaler()
        df[["SA_z", "SBSOD_z"]] = scaler.fit_transform(df[["SA_Score", "SBDS_score"]])
        print("✅ Standardized SA_Score & SBDS_score → SA_z, SBSOD_z")
    else:
        raise ValueError("Required columns for standardization not found: SA_Score, SBDS_score")

predictors = [
    "orientation_entropy_z",
    "degree_mean_z",
    "circuity_log_z",
    "SA_z",
    "SBSOD_z",
    "Gender_M",
]

required = predictors + ["tap_rate_z", "cv_iti_z", "partId"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns needed for Bayesian model: {missing}\nPresent: {list(df.columns)}")

# Complete cases only
mask = df[required].notna().all(axis=1)
df_clean = df.loc[mask, required].reset_index(drop=True)

# Center Gender_M
df_clean["Gender_M"] = df_clean["Gender_M"].astype(float) - df_clean["Gender_M"].astype(float).mean()

print(f"✅ Data ready | Sessions: {len(df_clean)} | Participants: {df_clean['partId'].nunique()}")
df_clean.head(3)


In [ ]:

# Build design matrices
Y = df_clean[["tap_rate_z", "cv_iti_z"]].to_numpy(dtype="float64")
X = df_clean[predictors].to_numpy(dtype="float64")

n_obs, n_pred = X.shape
n_out = Y.shape[1]
print("Shapes → X:", X.shape, "Y:", Y.shape, "| preds:", n_pred, "| outs:", n_out)


In [ ]:

with pm.Model() as mv_model:
    # Intercepts
    alpha = pm.Normal("alpha", mu=0.0, sigma=0.5, shape=(n_out,))

    # Coefficients (predictors × outcomes)
    B = pm.Normal("B", mu=0.0, sigma=0.5, shape=(n_pred, n_out))

    # LKJ prior on residual covariance
    L, corr, sigmas = pm.LKJCholeskyCov(
        "L",
        n=n_out,
        eta=2.0,
        sd_dist=pm.HalfNormal.dist(0.5)
    )
    Sigma = pm.Deterministic("Sigma", L @ L.T)

    # Mean matrix
    mu = alpha + at.dot(X, B)

    # Likelihood
    y_like = pm.MvNormal("y_like", mu=mu, chol=L, observed=Y)

    trace = pm.sample(
        draws=1500,
        tune=1500,
        chains=4,
        cores=1,          # stable on macOS/VS Code
        init="adapt_diag",
        target_accept=0.95,
        random_seed=42
    )

# Save trace
az.to_netcdf(trace, OUT_DIR / "trace_bayesmv.nc")
trace


In [ ]:

summary = az.summary(trace, var_names=["alpha","B","Sigma"], round_to=3)
summary_path = OUT_DIR / "summaries" / "posterior_summary.csv"
summary.to_csv(summary_path, index=True)
print(summary)
print("Saved summary →", summary_path)

# Residual correlation rho between tap_rate_z and cv_iti_z
Sigma_samps = trace.posterior["Sigma"].stack(s=("chain","draw")).values
cov_xy = Sigma_samps[0,1,:]
var_x  = Sigma_samps[0,0,:]
var_y  = Sigma_samps[1,1,:]
rho = cov_xy / np.sqrt(var_x * var_y)

rho_mean = float(rho.mean())
rho_ci = (float(np.percentile(rho, 2.5)), float(np.percentile(rho, 97.5)))

with open(OUT_DIR / "summaries" / "residual_correlation.txt", "w") as f:
    f.write("Residual correlation (tap_rate_z, cv_iti_z)\n")
    f.write(f"Posterior mean ρ = {rho_mean:.3f}\n")
    f.write(f"95% CrI = [{rho_ci[0]:.3f}, {rho_ci[1]:.3f}]\n")

print(f"\nResidual correlation Posterior mean ρ = {rho_mean:.3f}")
print(f"95% CrI: [{rho_ci[0]:.3f}, {rho_ci[1]:.3f}]")


In [ ]:

fig = az.plot_forest(trace, var_names=["alpha","B"], combined=True, hdi_prob=0.95)
plt.axvline(0, ls="--", c="black")
plt.title("Posterior Coefficients (95% CrI)")
plt.tight_layout()
fig_path = OUT_DIR / "figs" / "forest_coefficients.png"
plt.savefig(fig_path, dpi=300, bbox_inches="tight")
plt.show()
print("Saved →", fig_path)


In [ ]:

plt.figure(figsize=(8,5))
plt.hist(rho, bins=60, density=True)
plt.axvline(rho_mean, ls="--", c="black")
plt.xlabel("Residual correlation (ρ)")
plt.ylabel("Posterior density")
plt.title("Residual Correlation: Tap Rate vs CV–ITI")
plt.tight_layout()
rho_path = OUT_DIR / "figs" / "rho_posterior.png"
plt.savefig(rho_path, dpi=300, bbox_inches="tight")
plt.show()
print("Saved →", rho_path)
